In [23]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 13 20:12:24 2020

@author: bcbirkel
"""

## Cleaner plotting script

# %% IMPORT STATEMENTS
from obspy import read, read_inventory
from obspy.signal.filter import bandpass
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
from obspy.signal.invsim import simulate_seismometer, corn_freq_2_paz
from obspy.geodetics import gps2dist_azimuth
from obspy.io.sac import sacpz
from obspy.io.sac import attach_paz
from obspy.io.gse2.paz import read_paz
import os
from obspy.core import Trace, Stream, AttribDict
import math
from matplotlib import animation

#Ignore warnings due to python 2 and 3 conflict
import warnings
warnings.filterwarnings("ignore")

hello


In [24]:

# %% SET TOGGLE PARAMETERS
syn_exist = True
plotsyn = True
plot_azimuth = False
bpass = True
plot_spec_st = False
select_stns = False
skip_stns = True;
skip = 1
CI_only = False
CE_only = False
plotnames = True

event_no = 0

snr = 1

# plotting params
ymin = 0
ymax = 42
xmin = 0
xmax = 80

#set corners for map
llcrnrlon = -119
llcrnrlat = 33.25
urcrnrlon = -117
urcrnrlat = 34.75

#smaller map:
llcrnrlon = -118.5
llcrnrlat = 33.5
urcrnrlon = -117.5
urcrnrlat = 34.25

#set bandpass frequencies
high_freq = 1 / 2
low_freq = 1 / 10

In [25]:
# %% STATIONS PARAMETERS
# %% EVENT PARAMETERS AND METADATA
#set event

events = ['lahabra_2014', 'beverlyhills_2001', 'chatsworth_2007', 'chinohills_2008', 'inglewood_2009', 'elmonte_2020']
event = events[event_no]

if event_no == 0:
    event_title = 'lahabra'
    event_disp = 'La Habra 2014'
    event_lat = 33.9325
    event_lon = -117.9158
    gain = 20
    sy_gain = 10
elif event_no == 3:
    event_title = 'chinohills'
    event_disp = 'Chino Hills 2008'
    event_lat = 33.9465
    event_lon = -117.7667
    gain = 10
elif event_no == 4:
    event_title = 'inglewood'
    event_disp = 'Inglewood 2009'
    event_lat = 33.9377
    event_lon = -118.3357
    gain = 50
    sy_gain = 200
    vert_gain = 300
elif event_no == 2:
    event_title = 'chatsworth'
    event_disp = 'Chatsworth 2007'
    event_lat = 34.2983
    event_lon = -118.6255
    gain = 50
    sy_gain = 20
elif event_no == 1:
    event_title = 'beverlyhills'
    event_disp = 'Beverly Hills 2001'
    event_lat = 34.0541
    event_lon = -118.3929
    gain = 20
    sy_gain = 50
elif event_no == 5:
    event_title = 'elmonte'
    event_disp = 'El Monte 2020'
    event_lat = 34.0385
    event_lon = -118.0803
    gain_data = 5
    sy_gain = 20
    gain = 20
else:
    print('unknown event file')

In [26]:
sy_gain = gain
gain_data = gain
# quick fix to avoid renaming params later
evla = event_lat;
evlo = event_lon
evlat = evla;
evlon = evlo

In [19]:

# %% READ IN DATA AND SYNTHETICS
#set directories
path_dir_data = './CompiledEvents/' + event + '/allData/fixed/'
if syn_exist == True:
    path_dir_syn = './CompiledEvents/' + event + '/GravesSyn/CVM-S4/'

# read in data files
if event_no == 5:
    stream_dataN = read(path_dir_data + "*N_vel.SAC")
    stream_dataE = read(path_dir_data + "*E_vel.SAC")
    stream_dataZ = read(path_dir_data + "*Z_vel.SAC")
    stream_dataR = read(path_dir_data + "*R_vel.SAC")
    stream_dataT = read(path_dir_data + "*T_vel.SAC")
else:
    stream_dataN = read(path_dir_data + "*N_vel.SAC")  #+ read(path_dir_data + "*N.SAC")
    stream_dataE = read(path_dir_data + "*E_vel.SAC")  #+ read(path_dir_data + "*E.SAC")
    stream_dataZ = read(path_dir_data + "*Z_vel.SAC")  #+ read(path_dir_data + "*Z.SAC")
    stream_dataR = read(path_dir_data + "*R_vel.SAC")  #+ read(path_dir_data + "*R.SAC")
    stream_dataT = read(path_dir_data + "*T_vel.SAC")  #+ read(path_dir_data + "*T.SAC")

In [20]:

for tr in stream_dataN:
    cmp = "N"
    tr.stats.channel = "BH" + cmp
    tr.stats.sac.kcmpnm = "BH" + cmp
    tr.stats.sac.evlo = evlo
    tr.stats.sac.evla = evla
for tr in stream_dataE:
    cmp = "E"
    tr.stats.channel = "BH" + cmp
    tr.stats.sac.kcmpnm = "BH" + cmp
    tr.stats.sac.evlo = evlo
    tr.stats.sac.evla = evla
for tr in stream_dataZ:
    cmp = "Z"
    tr.stats.channel = "BH" + cmp
    tr.stats.sac.kcmpnm = "BH" + cmp
    tr.stats.sac.evlo = evlo
    tr.stats.sac.evla = evla
for tr in stream_dataR:
    cmp = "R"
    tr.stats.channel = "BH" + cmp
    tr.stats.sac.kcmpnm = "BH" + cmp
    tr.stats.sac.evlo = evlo
    tr.stats.sac.evla = evla
for tr in stream_dataT:
    cmp = "T"
    tr.stats.channel = "BH" + cmp
    tr.stats.sac.kcmpnm = "BH" + cmp
    tr.stats.sac.evlo = evlo
    tr.stats.sac.evla = evla

In [21]:

# read in synthetics files
if syn_exist == True:
    # set syn files path
    stream_synN = read(path_dir_syn + "Syn*N.SAC")
    stream_synE = read(path_dir_syn + "Syn*E.SAC")
    stream_synZ = read(path_dir_syn + "Syn*Z.SAC")
    stream_synR = read(path_dir_syn + "Syn*R.SAC")
    stream_synT = read(path_dir_syn + "Syn*T.SAC")

    for tr in stream_synN:
        cmp = "N"
        tr.stats.channel = "BH" + cmp
        tr.stats.sac.kcmpnm = "BH" + cmp
        tr.stats.sac.evlo = evlo
        tr.stats.sac.evla = evla
    for tr in stream_synE:
        cmp = "E"
        tr.stats.channel = "BH" + cmp
        tr.stats.sac.kcmpnm = "BH" + cmp
        tr.stats.sac.evlo = evlo
        tr.stats.sac.evla = evla
    for tr in stream_synZ:
        cmp = "Z"
        tr.stats.channel = "BH" + cmp
        tr.stats.sac.kcmpnm = "BH" + cmp
        tr.stats.sac.evlo = evlo
        tr.stats.sac.evla = evla
    for tr in stream_synR:
        cmp = "R"
        tr.stats.channel = "BH" + cmp
        tr.stats.sac.kcmpnm = "BH" + cmp
        tr.stats.sac.evlo = evlo
        tr.stats.sac.evla = evla
    for tr in stream_synT:
        cmp = "T"
        tr.stats.channel = "BH" + cmp
        tr.stats.sac.kcmpnm = "BH" + cmp
        tr.stats.sac.evlo = evlo
        tr.stats.sac.evla = evla

Exception: No file matching file pattern: ./CompiledEvents/lahabra_2014/GravesSyn/CVM-S4/Syn*N.SAC

In [ ]:

    # %% BUILD STREAMS FOR DATA AND SYNTHETICS
stream_data = stream_dataN + stream_dataE + stream_dataZ + stream_dataR + stream_dataT
if syn_exist == True:
    stream_syn = stream_synN + stream_synE + stream_synZ + stream_synR + stream_synT

sta_file = './all_stationmaster.txt'
file_st = open(sta_file, 'r')
line = file_st.readline()  # read first line to skip header information
lines = file_st.readlines()

# Load station coords into arrays, many more stations than used
st_num = []
st_netw = []
st_name = []
st_dist = []
st_az = []
st_baz = []
st_lat = []
st_lon = []
for line in lines:
    split_line = line.split()
    st_num.append(split_line[0])
    st_netw.append(split_line[2])
    st_name.append(split_line[3])
    st_lat.append(split_line[4])
    st_lon.append(split_line[5])
    distance = gps2dist_azimuth(evla, evlo, float(split_line[4]), float(split_line[5]))  # Get traveltime and azimuth
    #        print('Event ' + str(ev_lat) + ' ' + str(ev_lon) + ' station ' + split_line[4] + ' ' + split_line[5] + ' distance ' + str(distance[0]))
    st_dist.append(distance[0] / 1000.)  # distance
    st_az.append(distance[1])  # azimuth
    st_baz.append(distance[2])  # back-azimuth
print('number of stations in list is ' + str(len(st_num)))

for ii in range(len(st_name)):
    st = st_name[ii]
    for tr in stream_data:
        if tr.stats.station == st:
            tr.stats.sac.stla = float(st_lat[ii])
            tr.stats.sac.stlo = float(st_lon[ii])
            tr.stats.sac.evla = evla
            tr.stats.sac.evlo = evlo
            tr.stats.dist = st_dist[ii]

In [ ]:

# %% FIND STATIONS FOR COMPARISON

# initialize stn list
data_stns = []
syn_stns = []

# data stations
for tr in stream_data:
    if CI_only == True and tr.stats.network == "CI":
        data_stns.append(tr.stats.sac.kstnm)
        print("CI only - " + tr.stats.network + ', ' + tr.stats.sac.kstnm + ', ' + str(tr.stats.sac.stlo) + ', ' + str(
            tr.stats.sac.stla))
    elif CE_only == True and tr.stats.network == 'CE':
        data_stns.append(tr.stats.sac.kstnm)
        print("CE only - " + tr.stats.network + ', ' + tr.stats.sac.kstnm + ', ' + str(tr.stats.sac.stlo) + ', ' + str(
            tr.stats.sac.stla))
    elif CI_only == False and CE_only == False:
        data_stns.append(tr.stats.sac.kstnm)
        print("All stations - " + tr.stats.network + ', ' + tr.stats.sac.kstnm + ', ' + str(
            tr.stats.sac.stlo) + ', ' + str(tr.stats.sac.stla))

# synthetic stations
if syn_exist == True:
    for tr in stream_syn:
        syn_stns.append(tr.stats.station)

# find unique stations
data_st = set(data_stns)
if syn_exist == True:
    syn_st = set(syn_stns)

# build list with intersectioning stations:
if syn_exist == True:
    stn_list = data_st.intersection(syn_st)
if syn_exist == False:
    stn_list = data_st
print(stn_list)

In [ ]:

# %% GET DISTANCES FROM SOURCE TO STATIONS

for tr in stream_data:
    if tr.stats.sac.stlo > 0:
        tr.stats.sac.stlo = -tr.stats.sac.stlo  # fix longitude sign problem because map can't handle negatives
    distance = gps2dist_azimuth(tr.stats.sac.evla, tr.stats.sac.evlo, tr.stats.sac.stla,
                                tr.stats.sac.stlo)  # Get traveltime and azimuth
    tr.stats.dist = distance[0] / 1000.  #m to km
    tr.stats.az = distance[1]
    tr.stats.baz = distance[2]
    print(tr.stats.network + ", " + tr.stats.station + ": (" + str(tr.stats.sac.stla) + ", " + str(
        tr.stats.sac.stlo) + "), dist: " + str(tr.stats.dist))

if syn_exist == True:
    for tr in stream_syn:
        distance = gps2dist_azimuth(tr.stats.sac.evla, tr.stats.sac.evlo, tr.stats.sac.stla,
                                    tr.stats.sac.stlo)  # Get traveltime and azimuth
        tr.stats.dist = distance[0] / 1000.  #m to km
        tr.stats.az = distance[1]
        tr.stats.baz = distance[2]

In [ ]:

# %% PLOT STATIONS AND EVENT ON MAP

f = plt.figure(figsize=(10, 5))
# setup mercator map projection.
m = Basemap(projection='merc', llcrnrlon=llcrnrlon, llcrnrlat=llcrnrlat, urcrnrlon=urcrnrlon, urcrnrlat=urcrnrlat,
            epsg=4269)  #http://server.arcgisonline.com/arcgis/rest/services; EPSG Number of America is 4269
#m.arcgisimage(service='ESRI_Imagery_World_2D', xpixels = 2000, verbose= True)
basin_st = []

# plot stations
for tr in stream_data:
    if select_stns == True and tr.stats.station in stn_select or plotnames == True and tr.stats.dist < 30:
        stlat = tr.stats.sac.stla;
        stlon = tr.stats.sac.stlo
        xx, yy = m(stlon, stlat)
        m.scatter(xx, yy, marker="^", s=100, c="g", edgecolors="k", alpha=1, label="Data")
        plt.text(xx + .025, yy - 0.01, tr.stats.station, fontsize=8)
        if llcrnrlat < stlat < urcrnrlat and llcrnrlon < stlon < urcrnrlon:
            basin_st.append(tr.stats.sac.kstnm)
    # else:
    #     stlat = tr.stats.sac.stla; stlon = tr.stats.sac.stlo
    #     xx,yy = m(stlon,stlat)
    #     m.scatter(xx, yy, marker = "^" ,s=100, c="b" , edgecolors = "k", alpha = 1, label = "Data")
    #     if llcrnrlat < stlat < urcrnrlat and llcrnrlon < stlon < urcrnrlon:
    #         basin_st.append(tr.stats.sac.kstnm)

## for plotting syn stations - ignore because now using full grid
# if syn_exist == True:
#     for tr in stream_syn:
#         for st in basin_st:
#             if tr.stats.sac.kstnm == st:
#                 stlat = tr.stats.sac.stla; stlon = tr.stats.sac.stlo
#                 xx,yy = m(stlon,stlat)
#                 m.scatter(xx, yy, marker = "^" ,s=75, c="g" , edgecolors = "k", alpha = 1, label = "Graves Syn")

In [ ]:

#Plot the event
xx, yy = m(evlon, evlat)
m.scatter(xx, yy, marker="*", s=300, c="r", edgecolors="k", alpha=1)

if syn_exist == True:
    plt.title("Stations used for Data v. Synthetic Comparison - " + event_disp)
if syn_exist == False:
    plt.title("All available data stations - " + event_disp)
plt.show()

fig_path = './CompiledEvents/' + event + '/Figures/'
f.savefig(fig_path + "Station map - " + event)

In [ ]:

# %% SET UP SEISMOGRAM PLOTTING

# initialize
data_plot = Stream()
syn_plot = Stream()
data_plot_temp = Stream()
syn_plot_temp = Stream()
X = []
Y = []
stnpairs = []
stn_list = list(stn_list)  # turn intersecting station array to list
basin_st = set(basin_st)  # unique basin stations
done = []

# cut out stations past ymax and with low signal-to-noise
for tr in stream_data:
    for st in stn_list:
        amp = tr.max()
        amp_adjust = amp * (tr.stats.dist)
        if tr.stats.station == st and tr.stats.dist < ymax and abs(amp_adjust) > snr and st not in done:
            if select_stns == True and st in stn_select:
                stnpairs.append([st, tr.stats.dist])
                done.append(st)
                print(st + ", " + str(tr.stats.dist))
                print(amp_adjust)
            elif select_stns == False and st not in stn_ignore and st not in done:
                stnpairs.append([st, tr.stats.dist])
                done.append(st)
                print(st + ", " + str(tr.stats.dist))
                print(amp_adjust)


# get distance for sorting
def getDist(elem):
    return elem[1]


stnpairs.sort(key=getDist)
print(stnpairs)

stn_list_sorted = []
for pair in stnpairs:
    stn_list_sorted.append(pair[0])
print(stn_list_sorted)

# skip some stations IF using all stations, not selection
if select_stns == False and skip_stns == True:
    stn_plot = stn_list_sorted[::skip]
elif select_stns == True:
    stn_plot = stn_list_sorted

# CHECK that sort occured correctly:
done_stn = []
print("check that stations are sorted by distance:")
for stn in stn_plot:
    for tr in stream_data:
        if stn == tr.stats.station and stn not in done_stn:
            print("station name: " + stn + ", distance: " + str(tr.stats.dist))
            done_stn.append(stn)

        # set up flags for data v syn streams, normalize all
stream_norm = Stream()
for sta in stn_plot:  # for data and syn
    for tr in stream_data:
        if tr.stats.station == sta:
            tr.stats.label = "data"
            stream_norm += tr
    if syn_exist == True:
        for tr in stream_syn:
            if tr.stats.station == sta:
                tr.stats.label = "syn"
                stream_norm += tr
stream_norm.normalize(global_max=True)

# seperate normalized streams out again
for tr in stream_norm:
    if tr.stats.label == "data":
        data_plot += tr
    if tr.stats.label == "syn":
        syn_plot += tr

# CHECK that sort occured correctly:
done_stn = []
print("check that stations are sorted by distance AFTER NORMALIZATION:")
for stn in stn_plot:
    for tr in stream_data:
        if stn == tr.stats.station and stn not in done_stn:
            print("network: " + tr.stats.network + ", station name: " + stn + ", distance: " + str(tr.stats.dist))
            done_stn.append(stn)

In [ ]:

        # %% SET UP FIGURES FOR PLOTTING

# figures for distance plots
figN = plt.figure(0, figsize=[8, 5])
aN = figN.add_subplot(1, 1, 1)
figE = plt.figure(1, figsize=[8, 5])
aE = figE.add_subplot(1, 1, 1)
figR = plt.figure(2, figsize=[8, 5])
aR = figR.add_subplot(1, 1, 1)
figT = plt.figure(3, figsize=[8, 5])
aT = figT.add_subplot(1, 1, 1)
figZ = plt.figure(4, figsize=[8, 5])
aZ = figZ.add_subplot(1, 1, 1)

# # fig for combined plot
# figCmb = plt.figure(5, figsize=[24,5])
# aCmbA = figCmb.add_subplot(1, 1, 1)
# aCmbB = figCmb.add_subplot(1, 2, 2)
# aCmbC = figCmb.add_subplot(1, 3, 3)

# figures from azimuth plots
if plot_azimuth == True:
    figNaz = plt.figure(5, figsize=[8, 5])
    azN = figNaz.add_subplot(1, 1, 1)
    figEaz = plt.figure(6, figsize=[8, 5])
    azE = figEaz.add_subplot(1, 1, 1)
    figRaz = plt.figure(7, figsize=[8, 5])
    azR = figRaz.add_subplot(1, 1, 1)
    figTaz = plt.figure(8, figsize=[8, 5])
    azT = figTaz.add_subplot(1, 1, 1)
    figZaz = plt.figure(9, figsize=[8, 5])
    azZ = figZaz.add_subplot(1, 1, 1)
    figsAz = [figNaz, figEaz, figRaz, figTaz, figZaz]
    axesAz = [azN, azE, azR, azT, azZ]

if syn_exist == True:
    syn_times = np.arange(0, len(stream_syn.traces[0]) * stream_syn.traces[0].stats.delta,
                          stream_syn.traces[0].stats.delta)

for sytr in syn_plot:
    for tr in data_plot:
        if tr.stats.station == sytr.stats.station:
            sytr.stats.dist = tr.stats.dist

# %% PLOTTING SEISMOGRAMS

# set up directory to save figs
fig_path = './CompiledEvents/' + event + '/Figures/'
if not os.path.exists(fig_path):
    os.makedirs(fig_path)
os.chdir(fig_path)

# set up vars to loop through
figs = [figN, figE, figR, figT, figZ]
axes = [aN, aE, aR, aT, aZ]
channels = ["BHN", "BHE", "BHR", "BHT", "BHZ"]

# loop through each component
for i in range(len(axes)):
    fig = figs[i]
    a = axes[i]
    if plot_azimuth == True:
        figAz = figsAz[i]
        azA = axesAz[i]
    cha = channels[i]

    a.set_ylabel("Distance from source (km)")
    a.set_xlabel("Time (s)")
    a.set_ylim(ymin, ymax)
    a.set_xlim(xmin, xmax)
    a.set_title("Data (black) v. Syn (red) for " + event_disp + " - " + cha)

    for tr in data_plot:
        if tr.stats.station in stn_plot and tr.stats.channel == cha:

            # set up vars
            times = np.linspace(0, tr.stats.npts * tr.stats.delta, tr.stats.npts)
            dist = tr.stats.dist;
            azimuth = tr.stats.az;
            df = tr.stats.sampling_rate
            gain = gain_data * math.sqrt(dist);
            sy_gain = gain

            # bandpass
            if bpass == True:
                tr.filter("bandpass", freqmin=low_freq, freqmax=high_freq)
            if bpass == False:
                gain = gain / 10

            # plot data
            a.plot(times, tr.data * gain + dist, c="k", linewidth=0.5)

            # add station names
            if dist < ymax:
                a.text(xmax - 10, dist, tr.stats.sac.kstnm)

    if syn_exist == True and plotsyn == True:
        for sytr in syn_plot:
            if sytr.stats.station in stn_plot and sytr.stats.sac.kcmpnm == cha:
                # set up times for x-axis
                syn_times = np.linspace(0, sytr.stats.npts * sytr.stats.delta, sytr.stats.npts)
                # bandpass synthetics
                sytr.filter("bandpass", freqmin=low_freq, freqmax=high_freq)
                # plot synthetics on top of data
                a.plot(syn_times, sytr.data * sy_gain + sytr.stats.dist, c="r", linewidth=0.5)

    fig.savefig(event_disp + " Data by Distance - " + cha + ".png")

    #plot by azimuth
    if plot_azimuth == True:
        for tr in data_plot:
            if tr.stats.station in stn_select and tr.stats.channel == cha:
                #vars
                gain = gain_data * math.sqrt(dist)
                azfix = 360 / ymax

                # plot
                aztimes = np.linspace(0, tr.stats.npts * tr.stats.delta, tr.stats.npts)
                azA.plot(aztimes, tr.data * gain * azfix + azimuth, c="k", linewidth=0.5)
                azA.set_ylabel("Azimuth from source (degrees)")
                azA.set_xlabel("Time (s)")
                azA.set_xlim(xmin, xmax)
                azA.set_title("Data (black) v. Syn (red) for " + event_disp + " - North")
                azA.text(xmax - 10, azimuth, tr.stats.sac.kstnm)
        for tr in syn_plot:
            if tr.stats.station in stn_select and tr.stats.channel == cha:
                azA.plot(times, tr.data * gain * azfix + azimuth, c="r", linewidth=0.5)

        figAz.savefig(fig_path + event_disp + " Data by Aziumth - " + cha + ".png")

plt.show()

# loop through each component
a = plt.figure(10, figsize=[24, 5])
asR = fig.add_subplot(1, 1, 1)
asT = fig.add_subplot(1, 2, 2)
asZ = fig.add_subplot(1, 3, 3)

aR.set_ylabel("Distance from source (km)")
aT.set_title("Data (black) v. Syn (red) for " + event_disp + " - Radial, Transverse and Vertical")

subaxes = [asR, asT, asZ]
channels = ["BHR", "BHT", "BHZ"]

for i in range(len(subaxes)):
    for tr in data_plot:
        if tr.stats.station in stn_plot and tr.stats.channel == cha:

            # set up vars
            aS = subaxes[i]
            times = np.linspace(0, tr.stats.npts * tr.stats.delta, tr.stats.npts)
            dist = tr.stats.dist;
            azimuth = tr.stats.az;
            df = tr.stats.sampling_rate
            gain = gain_data * math.sqrt(dist);
            sy_gain = gain

            # bandpass
            if bpass == True:
                tr.filter("bandpass", freqmin=low_freq, freqmax=high_freq)
            if bpass == False:
                gain = gain / 10

            # plot data
            aS.plot(times, tr.data * gain + dist, c="k", linewidth=0.5)

            #add labels
            aS.set_xlabel("Time (s)")
            aS.set_ylim(ymin, ymax)
            aS.set_xlim(xmin, xmax)

            # add station names
            if dist < ymax:
                a.text(xmax - 10, dist, tr.stats.sac.kstnm)

    if syn_exist == True and plotsyn == True:
        for sytr in syn_plot:
            if sytr.stats.station in stn_plot and sytr.stats.sac.kcmpnm == cha:
                # set up times for x-axis
                syn_times = np.linspace(0, sytr.stats.npts * sytr.stats.delta, sytr.stats.npts)
                # bandpass synthetics
                sytr.filter("bandpass", freqmin=low_freq, freqmax=high_freq)
                # plot synthetics on top of data
                aS.plot(syn_times, sytr.data * sy_gain + sytr.stats.dist, c="r", linewidth=0.5)

    fig.savefig(event_disp + " Data by Distance - " + cha + ".png")
plt.show()